In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from libs.losses import  l2, SignalDiceLoss

In [2]:
sampling_rate = 1000
sdsc   = SignalDiceLoss(eps=0, soft=False)
sdsc_1 = SignalDiceLoss(eps=0, alpha=1)
sdsc_10 = SignalDiceLoss(eps=0, alpha=10)
sdsc_100 = SignalDiceLoss(eps=0, alpha=100)

In [3]:
def mse(a, b):
    return torch.mean((a-b)**2)

def mae(a, b):
    return torch.mean(torch.abs(a-b))

def dtw_distance_normalized(s1, s2):
    n, m = len(s1), len(s2)
    dtw_matrix = np.full((n + 1, m + 1), np.inf)
    dtw_matrix[0, 0] = 0

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = abs(s1[i - 1] - s2[j - 1])
            dtw_matrix[i, j] = cost + min(
                dtw_matrix[i - 1, j],    # insertion
                dtw_matrix[i, j - 1],    # deletion
                dtw_matrix[i - 1, j - 1] # match
            )

    # Traceback to count the path length
    i, j = n, m
    path_length = 0
    while i > 0 and j > 0:
        path_length += 1
        min_idx = np.argmin([
            dtw_matrix[i-1, j-1],
            dtw_matrix[i-1, j],
            dtw_matrix[i, j-1]
        ])
        if min_idx == 0:
            i -= 1
            j -= 1
        elif min_idx == 1:
            i -= 1
        else:
            j -= 1
    path_length += (i + j)  # finish remaining steps

    return dtw_matrix[n, m] / path_length
    
def make_signals(sampling_rate, noise_level=0.3, shift=1, scale_factor=0.5):
    t = torch.linspace(0,2*np.pi, sampling_rate) 
    gt = torch.sin(t).clone().detach().requires_grad_(True)
    signals = {
        "t" : t,
        "gt" : gt,
        "inverted": -gt,
        "scaled" :scale_factor * gt,
        "shift"  :gt+shift,
        "noisy":gt + noise_level * torch.randn(sampling_rate),
        "jittered":(gt + 0.05 * torch.randn_like(gt)),
    }
    return signals


def compute_grad_norm(loss_func, x, y):
    y = y.clone().detach().requires_grad_(True)
    loss = loss_func(x, y)
    loss.backward(retain_graph=True)
    print(loss)
    return y.grad.norm().item()

def print_grad_norm(x, y):
    print("MSE : ",compute_grad_norm(mse, x, y))
    print("MAE : ",compute_grad_norm(mae,  x, y))
    print("SDSC LOSS :",compute_grad_norm(sdsc,  x, y))
    print("SDSC Alpha 1 LOSS :",compute_grad_norm(sdsc_1,  x, y))
    print("SDSC Alpha 10 LOSS :",compute_grad_norm(sdsc_10,  x, y))
    print("SDSC Alpha 100 LOSS :",compute_grad_norm(sdsc_100,  x, y))

In [4]:
signal = make_signals(sampling_rate)

# Inverted

In [5]:
print_grad_norm(signal["gt"], signal["inverted"])

tensor(1.9980, grad_fn=<MeanBackward0>)
MSE :  0.08939799666404724
tensor(1.2720, grad_fn=<MeanBackward0>)
MAE :  0.03160696476697922
same_sign_mat min: 0.0
same_sign_mat max: 1.0
inputs == targets: False
sigmoid(inputs * targets * alpha): tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [6]:
print_grad_norm(signal["scaled"], -signal["scaled"])

tensor(0.4995, grad_fn=<MeanBackward0>)
MSE :  0.04469899833202362
tensor(0.6360, grad_fn=<MeanBackward0>)
MAE :  0.03160696476697922
same_sign_mat min: 0.0
same_sign_mat max: 1.0
inputs == targets: False
sigmoid(inputs * targets * alpha): tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [7]:
print_grad_norm(signal["gt"]*2, signal["inverted"]*2)

tensor(7.9920, grad_fn=<MeanBackward0>)
MSE :  0.17879599332809448
tensor(2.5439, grad_fn=<MeanBackward0>)
MAE :  0.03160696476697922
same_sign_mat min: 0.0
same_sign_mat max: 1.0
inputs == targets: False
sigmoid(inputs * targets * alpha): tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [8]:
print_grad_norm(signal["gt"], signal["gt"].roll(1))

tensor(1.9759e-05, grad_fn=<MeanBackward0>)
MSE :  0.00028113272855989635
tensor(0.0040, grad_fn=<MeanBackward0>)
MAE :  0.03162277862429619
same_sign_mat min: 0.0
same_sign_mat max: 1.0
inputs == targets: False
sigmoid(inputs * targets * alpha): tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1

# Shifted

In [9]:
print_grad_norm(signal["gt"], signal["shift"])

tensor(1., grad_fn=<MeanBackward0>)
MSE :  0.06324555724859238
tensor(1., grad_fn=<MeanBackward0>)
MAE :  0.03162277862429619
same_sign_mat min: 0.0
same_sign_mat max: 1.0
inputs == targets: False
sigmoid(inputs * targets * alpha): tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [10]:
print_grad_norm(signal["gt"], signal["shift"]-2)

tensor(1., grad_fn=<MeanBackward0>)
MSE :  0.06324555724859238
tensor(1., grad_fn=<MeanBackward0>)
MAE :  0.03162277862429619
same_sign_mat min: 0.0
same_sign_mat max: 1.0
inputs == targets: False
sigmoid(inputs * targets * alpha): tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [11]:
print_grad_norm(signal["gt"], signal["noisy"])

tensor(0.0915, grad_fn=<MeanBackward0>)
MSE :  0.01912841387093067
tensor(0.2378, grad_fn=<MeanBackward0>)
MAE :  0.03162277862429619
same_sign_mat min: 0.0
same_sign_mat max: 1.0
inputs == targets: False
sigmoid(inputs * targets * alpha): tensor([1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
        0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

# Jittered

In [12]:
print_grad_norm(signal["gt"], signal["jittered"])

tensor(0.0025, grad_fn=<MeanBackward0>)
MSE :  0.0031591744627803564
tensor(0.0396, grad_fn=<MeanBackward0>)
MAE :  0.03162277862429619
same_sign_mat min: 0.0
same_sign_mat max: 1.0
inputs == targets: False
sigmoid(inputs * targets * alpha): tensor([1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.